<a href="https://colab.research.google.com/github/guilmoreira/mercado_financeiro/blob/main/2_Indices_B3_setores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**1. INSTALANDO BIBLIOTECAS**



---


```
# Esta etapa irá instalar as bibliotecas necessárias para funcionamento do código
```



In [1]:
#!pip install -U plotly
#!pip install pandas
#!pip install numpy
!pip install kora -q

     |████████████████████████████████| 57 kB 2.1 MB/s 
     |████████████████████████████████| 60 kB 6.4 MB/s 


##**2. IMPORTANDO BIBLIOTECAS**

---


```
# Importando as bibliotecas
```




In [2]:
import pandas as pd
import numpy as np
import plotly.express as pl
import time
from time import sleep

#Usando selenium no google colab
from kora.selenium import wd
from datetime import date

#Validando certificado SSL. Não funciona sem esta biblioteca
import ssl


ssl._create_default_https_context = ssl._create_unverified_context

##**3. VARIÁVEIS INICIAIS**

---


```
# Variável de data utilizada devido ao nome do arquivo
```




In [3]:
#Data de Hoje
today=str(date.today())

##**4. ÍNDICES DA B3**

---


```
# LISTA COM PRINCIPAIS ÍNDICES DA B3
```



In [4]:
#LISTA DE ÍNDICES DA B3 -GITHUB
#indices_b3=pd.read_excel('https://github.com/guilmoreira/PythonProjects/blob/main/01_Indices_setores/indices_b3.xlsx?raw=true')
#indices_b3=pd.DataFrame(indices_b3)
#lista_indices=indices_b3['ID'].to_list()
lista_indices=['IBOV','SMLL']

In [7]:
indices_b3=lista_indices
indices_b3

['IBOV', 'SMLL']

##**5.FUNÇÃO EXTRAINDO TICKERS POR ÍNDICE**

---



```
# FUNÇÃO QUE EXTRAÍ A CARTEIRA HIPOTÉTICA PARA O ÍNDICE NAQUELA DATA
# NECSSÁRIO ATUALIZAR O "find_element"
```




5.1 **Referências utilizadas (03/28/2022)**

https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/indice-ibovespa-ibovespa-composicao-da-carteira.htm

https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br


In [8]:
#Função baixa os arquivos do site

def composicao_indice(indice, delay=10):
#funçao utiliza o selenium para realizar essa tarefa
  
  link=f'https://sistemaswebb3-listados.b3.com.br/indexPage/day/{indice.upper()}?language=pt-br'
  
  wd.get(link)

#ATUALIZAR O find_element_by_id para find_element(by=By.ID)?? Pesquisar!
  sleep(5)
  wd.find_element_by_id('segment').send_keys('Setor de Atuação')
  sleep(delay)
  wd.find_element_by_link_text('Download').click()
  sleep(5)
  arquivo=f'/content/{indice.upper()}Dia_{today[8:]}-{today[5:7]}-{today[2:4]}.csv'
  return pd.read_csv(arquivo, sep=';',encoding='ISO-8859-1',skipfooter=2,engine='python',thousands='.',decimal=',',header=1,index_col=False)

In [ ]:
#Função sem download dos arquivos precisa ajuste

def composicao_indice_no_file(indice, delay=10):
  url = f'https://sistemaswebb3-listados.b3.com.br/indexPage/day/{indice.upper()}?language=pt-br'
  return pd.read_html(url, decimal=',',thousands='.',index_col='Código')

##7. **LOOP PARA EXTRAÇÃO + ARQUIVOS**

---



```
# LOOP PARA DOWNLOAD DOS ARQUIVOS PARA CADA UM DOS ÍNDICES.
# GERADA UMA DATABASE (geral) com as carteiras
```



In [ ]:
#EXTRAÇÃO SOMENTE UM ÍNDICE
indice_unico='IBOV'
ibov=composicao_indice(str(indice_unico))

In [9]:
#EXTRAINTO TODOS OS ÍNDICES

tempo_total = time.time()

geral=[]

for i in lista_indices:
  tempo_ini = time.time()
  dados=composicao_indice(str(i))
  tempo_final  = time.time()
  tempo_ind=tempo_final-tempo_ini
  geral.append(dados)
  print(f'O índice {i} foi baixado com sucesso num tempo de {str(tempo_ind)} s. Dimensões: {dados.shape}')

tempo_loop=(time.time())-tempo_total

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  


O índice IBOV foi baixado com sucesso num tempo de 22.712398052215576 s. Dimensões: (91, 7)
O índice SMLL foi baixado com sucesso num tempo de 20.596530437469482 s. Dimensões: (136, 7)


##8. **TRATANDO OS DADOS**

---


```
# NECESSÁRIO AJUSTE NOS DADOS INDIVIDUALIZADOS
# DEIXEI UM PADRÃO PARA O IBOV
```



In [26]:
dados

,Setor,Código,Ação,Tipo,Qtde. Teórica,Part. (%),Part. (%)Acum.,SubSetor
0,Bens Industri,AERI3,AERIS,ON NM,218058827,0.380,2.755,Bens Industria
1,Bens Industri,ARML3,ARMAC,ON NM,170115256,0.948,2.755,Bens Industria
2,Bens Industri,ROMI3,INDS ROMI,ON NM,37272660,0.159,2.755,Bens Industria
3,Bens Industri,KEPL3,KEPLER WEBER,ON,30324161,0.484,2.755,Bens Industria
4,Bens Industri,MILS3,MILLS,ON NM,113954358,0.286,2.755,Bens Industria
...,...,...,...,...,...,...,...,...
131,Utilidade Pública,AURE3,AUREN,ON NM,281056116,1.403,10.457,Utilidade Públi
132,Utilidade Pública,ENBR3,ENERGIAS BR,ON ED NM,230931405,1.652,10.457,Utilidade Públi
133,Utilidade Pública,LIGT3,LIGHT S/A,ON NM,372555324,1.159,10.457,Utilidade Públi
134,Utilidade Pública,MEGA3,OMEGAENERGIA,ON NM,285136278,1.067,10.457,Utilidade Públi


In [10]:
#VERIRICANDO DISCREPÂCIAS
def renomear_setor(setor):
  if setor == 'Cons N  Básico' or setor=='Cons N Cíclico': return 'Consumo Não-Cíclico'
  if setor == 'Financeiro e Outros' or setor=='Financ e Outros': return 'Financeiro'
  if setor == 'Telecomunica' : return 'Telecomunicação'
  if setor == 'Utilidade Públ' : return 'Utilidade Pública'
  if setor == 'Bens Indls' : return 'Bens Industriais'
  if setor == 'Mats Básicos' : return 'Materiais Básicos'
  if setor == 'Petróleo' : return 'Petróleo'  
  if setor == 'Saúde' : return 'Saúde'  
  if setor == 'Tec.Informação' : return 'Tecnologia da Informação'  
  if setor == 'Telecomunicaçã' : return 'Comunicação'
  if setor == 'Diverso' : return 'Diversos'  
  if setor == 'Consumo Cíclico' : return 'Consumo Cíclico'
  if setor == 'Comput e Equips' : return 'Comput e Equips'
  else: return setor

def renomear_subsetor(subsetor):
  if subsetor == 'Máqs e Equips' : return 'Máquinas e Equipamentos'
  if subsetor == 'Mat Transporte' : return 'Material de Transporte'
  if subsetor == 'Comput e Equips' : return 'Computadores e Equipamentos'
  if subsetor == 'Comércio Distr.' : return 'Comércio e Distribuição'
  if subsetor == 'Pr Pessoal Limp' : return 'Produtos Limpeza'
  if subsetor == 'Tecid Vest Calç' : return 'Tecido, Vestuário e Calçados'
  if subsetor == 'Constr Civil' : return 'Construção Civil'
  if subsetor == 'Explor Imóveis' : return 'Exploração de Imóveis'
  if subsetor == 'Interms Financs' : return 'Intermediários Financeiro'
  if subsetor == 'Previd  Seguros' : return 'Previdência e Seguros'
  if subsetor == 'Sid Metalurgia' : return 'Siderurgia e Metalurgia'
  if subsetor == 'SM Hosp An.Diag' : return 'Serviços Médicos - Hospitalares, Análises e Diagnósticos'
  if subsetor == 'Programas Servs' : return 'Programas e Serviços'  
  if subsetor == 'Energ Elétrica' : return 'Energia Elétrica'
  if subsetor == 'Gás e Biocombustíveis' : return 'Petróleo, Gás e Biocombustíveis'
  if subsetor == 'Alimentos Processados' : return 'Alimentos Processados'
  if subsetor == 'Transportes' : return 'Transporte'
  if subsetor == 'Viagens e Lazer' : return 'Viagens e Lazer'
  if subsetor == 'Madeira e Papel' : return 'Madeira e Papel'
  if subsetor == 'Serviços Financeiros Diversos' : return 'Serviços Financeiros Diversos'
  if subsetor == 'Mineração' : return 'Mineração'
  if subsetor == 'Água Saneamento' : return 'Água Saneamento'  
  if subsetor == 'Telecomunicação' : return 'Telecomunicação'  
  if subsetor == 'Comércio' : return 'Comércio'
  if subsetor == 'Químicos' : return 'Químicos'
  if subsetor == 'Bebidas' : return 'Bebidas'
  if subsetor == 'Transporte' : return 'Transporte'     
  if subsetor == 'Diversos' : return 'Diversos'
  else: return subsetor

In [ ]:
#LOOP PARA TRATAR OS DADOS
carteira=[]
tempo_ini = time.time()

for i in geral:
  tempo_ini = time.time()
  tratamento=i
  #Criando uma coluna de subsetor partindo da barra
  tratamento['SubSetor']=tratamento['Setor'].apply(lambda s: s[s.rfind('/')+1:].strip())
  #Subscrevendo a coluna de Setor com dados antes da barra
  tratamento['Setor']=tratamento['Setor'].apply(lambda s: s[:s.rfind('/')].strip())
  #Aplicando função
  tratamento['Setor']=tratamento['Setor'].apply(renomear_setor)
  tratamento['SubSetor']=tratamento['SubSetor'].apply(renomear_subsetor)
  carteira.append(tratamento)
  print(f'numero {i}')

tempo_trat=tempo_ini-(time.time())

print(f' O tempo do loop foi de {str(tempo_trat)} s')

In [ ]:
carteira[0]

##9. **GRÁFICOS**

---



```
# COMO DADOS NÃO ESTÃO AJUSTADOS PARA TODOS OS ÍNDICES
# ALGUNS ÍNDICES PODEM NÃO FUNCIONAR
```



In [28]:
#lista dos indices
a=0
print('Indice numérico utilizado na figura')
for i in lista_indices:
  print(f'{i.upper()}={a}')
  a=a+1

Indice numérico utilizado na figura
IBOV=0
SMLL=1


In [32]:
#UTILIZAR O ÍNDICE DESEJADO
base=carteira[0]

In [33]:
#GRÁFICO DE PIZZA ITERATIVO TENDO QUE AJUSTAR CONFORME DADOS

fig1=pl.sunburst(data_frame=base,path=['Setor','SubSetor','Código'], values='Part. (%)', height=1000)
fig1.update_traces(textfont_color='white',
                  textfont_size=14,
                  hovertemplate='<b>%{label}:</b> %{value:.2f}%')

In [ ]:
#TREEMAP

fig2=pl.treemap(data_frame=base,path=['Setor','SubSetor','Código'], values='Part. (%)', height=1000)
fig2.update_traces(textfont_color='white',
                  textfont_size=14,
                  hovertemplate='<b>%{label}:</b> %{value:.2f}%')